In [7]:
from sedaro import SedaroApiClient
from pprint import pprint
API_KEY = "PKDqMrtcTK4plJgL7qVlQD.0xQph1lyKzd-pV0-ZL7bRIH7BX54Yjqbz6tluwut3Hvp8XE-RVbfHSz2o5vC77scUhg2xBFuBybplxY6FyXXMQ"
TEMP_AGENT_REPO_BRANCH_ID = "PRkHJlBrpwLDCqPqfbkdrb" # Specific Repo of the Templated agent 
SCENARIO_BRANCH_VERSION_ID = "PRx5rSwrGfkK4n9vFXmVbt" # Version 2
WORKSPACE_ID = "PQtnGZNNPdzZM5JdVhP5P9" # Violet/Ethreal workspace

In [38]:
# Define modules
sedaro = SedaroApiClient(api_key = API_KEY)
# agent template is only for satellite model (agent), and don't copy the model repo ID, only the branch ID (version)
agent_template_branch = sedaro.agent_template(TEMP_AGENT_REPO_BRANCH_ID) # we are going to change this templated agent

In [44]:
block_schema = agent_template_branch.data
pprint(block_schema)
# print(block_schema['id'])

{'_abstractBlockTypes': {'ActivePointingMode': False,
                         'AngleBetweenVectors': False,
                         'AngularVelocitySensor': False,
                         'Antenna': False,
                         'AreaTarget': False,
                         'AveragingAlgorithm': False,
                         'BatteryCell': False,
                         'BatteryPack': False,
                         'BmskBodyMask': False,
                         'BodyFrameVector': False,
                         'BodyMask': False,
                         'BusRegulator': False,
                         'CelestialTarget': False,
                         'CelestialVector': False,
                         'CircularFieldOfView': False,
                         'Clock': False,
                         'CombinationalLogic': False,
                         'Component': False,
                         'ComponentToScalarCondition': False,
                         'CompoundCondition': F

In [ ]:
def clone_template_as_new_repo(
    api_key: str,
    workspace_id: str,
    block_schema: dict,
    template_name: str,
):
    # 1) spin up a new Spacecraft repo + branch
    client = SedaroApiClient(api_key=api_key)
    repo = client.Repository.create(
        name=template_name,
        metamodelType="Spacecraft",
        workspace=workspace_id,
    )

    # 2) wrap its branch as an AgentTemplateBranch
    new_branch_id = repo.branches[0].id
    branch = client.agent_template(new_branch_id)

    # 3) build a map of (type,name)→id for all the default blocks already in the new branch
    existing = branch.data["blocks"]
    existing_by_type_name = {
        (blk["type"], blk.get("name")): bid
        for bid, blk in existing.items()
    }



    # 4) build our ref_map: old_id → either the real new-branch ID (for defaults)
    #    or a “$old_id” placeholder (for everything else)
    ref_map = {}
    for old_id, blk in existing.items():
        key = (blk["type"], blk.get("name"))
        if key in existing_by_type_name:
            # reuse this default
            ref_map[old_id] = existing_by_type_name[key]
        else:
            # create afresh
            ref_map[old_id] = f"${old_id}"

    delete_items = []
    for old_id, blk in existing.items():
        ref_id = ref_map[old_id]
        if not isinstance(ref_id, str) or not ref_id.startswith("$"):
            # we’re reusing an existing block → skip creation
            continue

        # build your new block entry
        b = {"id": ref_id, "type": blk["type"]}
        for k, v in blk.items():
            if k in ("id", "type"):
                continue

            # remap any string or list of strings that point at other old_ids
            if isinstance(v, str) and v in ref_map:
                b[k] = ref_map[v]
            elif (
                isinstance(v, list)
                and all(isinstance(x, str) and x in ref_map for x in v)
            ):
                b[k] = [ref_map[x] for x in v]
            else:
                b[k] = v

        delete_items.append(b)

    ref_map = {}
    for old_id, blk in block_schema.items():
        key = (blk["type"], blk.get("name"))
        if key in existing_by_type_name:
            # reuse this default
            ref_map[old_id] = existing_by_type_name[key]
        else:
            # create afresh
            ref_map[old_id] = f"${old_id}"

    # 5) turn block_schema → a single payload array, skipping any old_id that
    #    mapped to an existing ID (i.e. those defaults we’re reusing)
    payload = []
    for old_id, blk in block_schema.items():
        ref_id = ref_map[old_id]
        if not isinstance(ref_id, str) or not ref_id.startswith("$"):
            # we’re reusing an existing block → skip creation
            continue

        # build your new block entry
        b = {"id": ref_id, "type": blk["type"]}
        for k, v in blk.items():
            if k in ("id", "type"):
                continue

            # remap any string or list of strings that point at other old_ids
            if isinstance(v, str) and v in ref_map:
                b[k] = ref_map[v]
            elif (
                isinstance(v, list)
                and all(isinstance(x, str) and x in ref_map for x in v)
            ):
                b[k] = [ref_map[x] for x in v]
            else:
                b[k] = v

        payload.append(b)

    # 6) one atomic patch: create all our non‐default blocks in one go
    resp = branch.update(
        delete=delete_items,
        blocks=payload,
        include_response=True
    )

    print("✅ Created/updated:", resp["crud"]["blocks"])
    return branch
# usage
new_branch = clone_template_as_new_repo(
    api_key=API_KEY,
    workspace_id=WORKSPACE_ID,
    block_schema=agent_template_branch.data["blocks"],
    template_name="Clone"
)





✅ Created/updated: ['NSghWfrUj9OyAK8OBAXa-', 'NSghYm2RqDibih0igHFc-', 'NSghaQGUD5XkZv-hq4TTV', 'NSghcbxRIX9eXogEB0li-', 'NSgheitRQ51bMoH_95sZk', 'NSghgp0ThH8gNv5Ady03k', 'NSghpotSFHj43o7HNUfo-', 'PLCZKQWpjfNLcRbLg65yw6', 'PS2RplQSj4BWmhzMmPjhcw', 'PS2RplQVhDb66QvJmsYCLW', 'PS2RplQXbXTTTvk39J6CTw', 'PS2RplQZb3WgRT4M5GdMxL', 'PS2RplQcYr6NvVYKWhqzyr', 'PS2RplQfddhqtNpYBD8cYb', 'PS2RplQhbq9mV4CQYJ484g', 'PS2RplQkX5MLSMlQp2Yx9l', 'PS2RplQmYcz85mRZRnhsn4', 'PS2RplQpSdSzqNm3fC2ncl', 'PS2RplQrTgjfmtVf2R5jwV', 'PS2RplQtQkHnDQRmZfytDQ', 'PS2RplQwSsF2nJMYRQYqjl', 'PS2RplQyTpHCMHkTCtPrXh', 'PS2RplR2NgZCM8HJz8vfC8', 'PS2RplR4RkSYB57M7Rwrzj', 'PS2RplR6NGYpbWNWvsfz5g', 'PS2RplR8Mlg7t4RchpfCGP', 'PS2RplRBHpkr52zqmLq4Py', 'PS2RplRDKxzT3cz5mYhBT2', 'PS2RplRGKGyzHdcYdSvMdD', 'PS2RplRJKhvy2r29j9RzZg', 'PS2RplRLFKmXZBr2xy474m', 'PS2RplRNGKcBFCdJrPkzdP', 'PS2RplRQHZ8kRBTvyjZdMR', 'PS2RplRSCLhKBQhFQtHL6G', 'PS2RplRVF7NL3RWQlL79sw', 'PS2RplRXBH8vBwbkxK8BSw', 'PS2RplRZ8m9X7MxYjppGvG', 'PS2RplRc5vNmd8SfqM8tt7',

In [ ]:
print(agent_template_branch.data.keys())

dict_keys(['_abstractBlockTypes', '_blockNames', '_quantityKinds', '_relationships', '_supers', 'activeCommInterfaces', 'activePointingMode', 'activeRoutines', 'angularAcceleration', 'angularMomentum', 'angularMomentumMagnitude', 'angularVelocity', 'angularVelocityError', 'angularVelocitySolution', 'angularVelocitySolutionError', 'annotations', 'attitude', 'attitudeError', 'attitudeSolution', 'attitudeSolutionError', 'attitudeSolutionErrorAngle', 'beta', 'blocks', 'cadFileName', 'cadKey', 'cadScaleFactor', 'cadSignedUrl', 'cdhTimeStepLimiter', 'commandedAngularRates', 'commandedAttitude', 'definitiveAngularVelocitySolution', 'definitiveAngularVelocitySolutionError', 'definitiveAttitudeSolution', 'definitiveAttitudeSolutionError', 'definitiveAttitudeSolutionErrorAngle', 'definitivePositionSolution', 'definitivePositionSolutionError', 'definitiveVelocitySolution', 'definitiveVelocitySolutionError', 'dragTorque', 'dynamicCenterOfMass', 'dynamicInertia', 'dynamicMass', 'earthCoverage', 'ea

In [ ]:
## future plan
# figure out how to prompt schema to drop all the keys and things we can change
# sysMl to schema round trip 